# Dashboard notebook to control shared dask cluster 

In [1]:
from dask_jobqueue import PBSCluster
from distributed import Client

cluster = PBSCluster(project='P48500028',
                     queue='premium',
                     walltime='02:00:00')

client = Client(cluster)

# cluster.scale(4)  # TODO: figure out why we need to scale up to 1 worker to open the bcsd/maurer datasets

In [2]:
cluster

In [3]:
cluster.adapt(minimum=36, maximum=90, interval='500ms')

In [4]:
client

Client Scheduler: tcp://10.148.10.15:37891 Dashboard: proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
import json
import os
with open(os.path.join(os.environ['HOME'], 'scheduler_file.json'), 'w') as f:
    json.dump(client.scheduler_info(), f, indent=2)


In [6]:
import xarray as xr
xr.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.6.7 | packaged by conda-forge | (default, Feb 28 2019, 09:07:38) 
[GCC 7.3.0]
python-bits: 64
OS: Linux
OS-release: 4.12.14-94.41-default
machine: x86_64
processor: x86_64
byteorder: little
LC_ALL: None
LANG: None
LOCALE: en_US.UTF-8
libhdf5: 1.8.18
libnetcdf: 4.4.1.1

xarray: 0.11.3
pandas: 0.24.1
numpy: 1.16.3
scipy: 1.2.1
netCDF4: 1.3.1
pydap: None
h5netcdf: 0.7.0
h5py: 2.8.0
Nio: None
zarr: None
cftime: 1.0.3.4
PseudonetCDF: None
rasterio: None
cfgrib: None
iris: None
bottleneck: 1.2.1
cyordereddict: None
dask: 1.1.2
distributed: 1.26.0
matplotlib: 2.2.3
cartopy: 0.16.0
seaborn: 0.9.0
setuptools: 40.8.0
pip: 19.0.3
conda: None
pytest: None
IPython: 7.3.0
sphinx: None


In [7]:
print(cluster.job_script())

#!/bin/bash

#!/usr/bin/env bash
#PBS -N dask-worker
#PBS -q premium
#PBS -A P48500028
#PBS -l select=1:ncpus=36:mem=109GB
#PBS -l walltime=02:00:00
JOB_ID=${PBS_JOBID%.*}

export OMP_NUM_THREADS=1

/glade/u/home/jvano/miniconda/envs/loca/bin/python -m distributed.cli.dask_worker tcp://10.148.10.15:37891 --nthreads 4 --nprocs 9 --memory-limit 12.11GB --name dask-worker--${JOB_ID}-- --death-timeout 60

